In [34]:
import json
from itertools import product
from pathlib import Path

from evals.data import get_jsonl

In [26]:
cwd = Path.cwd()

winelist_relative_path = Path("registry/data/sommelia/winelists.jsonl")
criteria_relative_path = Path("registry/data/sommelia/criteria.jsonl")

winelist_path = cwd / winelist_relative_path
criteria_path = cwd / criteria_relative_path

In [27]:
winelists = get_jsonl(winelist_path)
criteria = get_jsonl(criteria_path)

In [30]:
samples = [{**winelist, **criterion} for winelist, criterion in product(winelists, criteria)]

In [35]:
def write_to_jsonl(data, file_name:str):
    path = cwd / Path(f"registry/data/sommelia/{file_name}.jsonl")
    with open(path, "a") as f:
        for line in data:
            json.dump(line, f, ensure_ascii=False)
            f.write("\n")

In [36]:
write_to_jsonl(samples, "samples2")